In [20]:
import multiprocessing
import os
import numpy as np
import json
import math
import threading

In [21]:
movies_path = os.path.join(os.path.join(os.getcwd(),'Datasets'),'movies_dataset')
books_path = os.path.join(os.getcwd(),'Book-Crossing dataset cleaning')

user_based_dict_path = os.path.join(movies_path,'utility_matrix_user_based.txt')
item_based_dict_path = os.path.join(movies_path,'utility_matrix_item_based.txt')

explicit_dict_path_books = os.path.join(books_path,'Explicit.csv')

implicit_dict_path_books = os.path.join(books_path,'Implicit.csv')


CORES = multiprocessing.cpu_count()
cliques = {}

In [22]:
'''
with open(user_based_dict_path) as fr:
    item_based_utility = json.load(fr)
'''   
 
""" with open(user_based_dict_path) as fr:
    user_based_utility = json.load(fr) """

with open(explicit_dict_path_books) as f:
    explicit_user_based_utility = {}
    
    fields = f.readline().strip().split('\t') # first line has no data

    item_ISBN_unique_index = fields.index('uniqueISBN')+1
    user_id_index = fields.index('userId')+1
    rating_index = fields.index('bookRating')+1
    print(user_id_index)
    while(True):
        line = f.readline().strip().split('\t')
        if(line == ['']):
            break

        item_name = line[item_ISBN_unique_index]
        user_id = line [user_id_index]
        rating = float(line[rating_index])

        if(explicit_user_based_utility.get(user_id) == None):
            explicit_user_based_utility[user_id] = {}
    
        explicit_user_based_utility[user_id][item_name] = rating    


with open(implicit_dict_path_books) as f:
    implicit_user_based_utility = {}
    
    fields = f.readline().strip().split('\t') # first line has no data

    item_ISBN_unique_index = fields.index('uniqueISBN')+1
    user_id_index = fields.index('userId')+1
    rating_index = fields.index('bookRating')+1

    while(True):
        line = f.readline().strip().split('\t')
        if(line == ['']):
            break

        item_name = line[item_ISBN_unique_index]
        user_id = line [user_id_index]
        rating = float(line[rating_index])
    
        if(implicit_user_based_utility.get(user_id) == None):
            implicit_user_based_utility[user_id] = {}
    
        implicit_user_based_utility[user_id][item_name] = rating    



1


In [23]:
"""
    converts the ratings and the keys of the dictionary
    {key1:{key2:rating}} into float

    user_based_utility -> {key1:{key2:rating}}
"""
def convert_ratings_to_float(user_based_utility):
    user_based_correct = {}

    for key1 in user_based_utility.keys():
        for key2 in user_based_utility[key1].keys():

            if('X' == key1[-1] or 'x' == key1[-1]):
                key11 = key1[:-1] + '1'
            elif( key1 == 'B00009ANY9'):
                key11 = '1'
            else:
                key11 = key1 + '0'
            
            if('X' == key2[-1] or 'x' == key2[-1]):
                key22 = key2[:-1] + '1'
            elif( key2 == 'B00009ANY9'):
                key22 = '1'
            else:
                key22 = key2 + '0'

            key11 = float(key11)
            key22 = float(key22)

            if(user_based_correct.get(key11) == None):
                user_based_correct[key11] = {}
            
            user_based_correct[key11][key22] = float(user_based_utility[key1][key2])
    
    return user_based_correct

#user_based_utility = convert_ratings_to_float(user_based_utility) #movies
explicit_user_based_utility = convert_ratings_to_float(explicit_user_based_utility)
implicit_user_based_utility = convert_ratings_to_float(implicit_user_based_utility)

In [24]:
# items are casted to float

# items = np.array(list(item_based_utility.keys())).astype(np.int)

# items

In [25]:
"""
    Returns a partition of dataset
    into num_folds dictionaries

    the return is [{key1:{key2:rating}}]
"""
def divide_dataset(dataset, num_folds):
    folds = [{}]*num_folds

    for key1 in dataset.keys():
        for key2 in dataset[key1].keys():
            rand_index = np.random.randint(num_folds)
            
            tmp_dict = folds[rand_index]

            if(tmp_dict.get(key1) == None):
                tmp_dict[key1] = {}
            
            tmp_dict[key1][key2] = dataset[key1][key2]
    
    return folds

num_folds = 4
#folds = divide_dataset(user_based_utility, num_folds) #movies
folds_explicit = divide_dataset(explicit_user_based_utility, num_folds)
folds_implicit = divide_dataset(implicit_user_based_utility, num_folds)


In [26]:
def merge_dicts(dicts):
    ret = {}

    for i in range(len(dicts)):
        tmp_dict = dicts[i]
        for key1 in tmp_dict.keys():
            for key2 in tmp_dict[key1].keys():

                if(ret.get(key1)==None):
                    ret[key1] = {}
                ret[key1][key2] = tmp_dict[key1][key2]

    return ret

In [27]:
""" train_dict = merge_dicts([fold for fold in folds[:3]])
test_dict = folds[3]  """ #movies


train_dict_explicit = merge_dicts([fold for fold in folds_explicit[:3]])
test_dict = folds_explicit[3] 

train_dict_implicit = merge_dicts([fold for fold in folds_explicit[:3]])
 

In [28]:
"""
    Calculates the average value of the 
    valeues in a

    a -> dictionary of non zero values 
"""
def avg(a):
    i=0
    tot = 0
    for k in a.keys():
        i +=1
        tot += a.get(k)
    
    return tot/i

"""
    Calculates a new dictionary that for each
    non null value of a has that values - const

    returns a - const

    a -> dictioary of non zero values
"""
def scale(a,const):
    ret = {}
    
    for k in a.keys():
        ret[k] = a.get(k) - const
    
    return ret

"""
    calculates the norm of a dictionary

    a -> dictioary of non zero values
"""
def norm (a):
    tot = 0
    for k in a.keys():
        tot += pow(a.get(k),2) 

    return math.sqrt(tot)

"""
    computes the inner product of two dictionaries

    a -> dictioary of non zero values
    b -> dictioary of non zero values

"""
def inner_product(a,b):
    tot = 0
    for k in a.keys():
        b_tmp = b.get(k)
        if(b_tmp != None):
            tot += b_tmp * a.get(k) 

    return tot

In [29]:
"""
    Calculates the correlation coefficent
        between two vectors
    
    a, b dictionaries of non zero values

    esplicit -> boolean value that  defines if we
    need to scale or not cause in the case of implicit
    we do not need to scale
"""
def compute_correlation_coefficent(a,b, is_explicit):

    a_scaled = a
    b_scaled = b

    if(is_explicit):
        avg_a = avg(a)
        avg_b = avg(b)

        a_scaled = scale(a, avg_a)
        b_scaled = scale(b, avg_b)

    a_scaled_norm = norm(a_scaled)
    b_scaled_norm = norm(b_scaled)

    if(a_scaled_norm == 0 or b_scaled_norm == 0):
        print('One of the two vectors has 0 norm, returning 0.')
        return 0 

    sim = inner_product(a_scaled,b_scaled)/(a_scaled_norm*b_scaled_norm)

    return sim

In [30]:
class ComputeCorrelationCoefficentThread (threading.Thread):
   """
      user_dict -> dictionary of the user: {itemID:rating}
      user_ids -> set of users IDs
      utility_matrix -> dataset
      is_explicit -> boolean value that  defines if we
         need to scale or not cause in the case of implicit
         we do not need to scale
   """
   def __init__(self, user_dict, user_ids, utility_matrix, is_explicit, name = None):
      threading.Thread.__init__(self)
      self.name = name
      self.user_dict = user_dict
      self.user_ids = user_ids
      self.is_explicit = is_explicit
      self.utility_matrix = utility_matrix
      self.result = None
   
   def run(self):
      if self.name == None:
         raise Exception('Something bad happened, thread has no name')
      print('Thread {} started'.format(self.name))
      self.compute_similarities()

   
   def compute_similarities(self):
      similarities = []
      for user in self.user_ids:
         tmp_user_dict = self.utility_matrix[user]
         similarity = compute_correlation_coefficent(self.user_dict, tmp_user_dict, self.is_explicit)
         similarities.append([user,similarity])
      
      self.result = similarities

   # vector of similarities --> [userID, sim_score]
   def join(self):
        threading.Thread.join(self)
        if self.result is not None:
            return self.result
        else:
            print('Error using threads, result is None')

In [31]:
"""
    returns a list containing num_folds lists 
    that are a partition of data

    data -> list
    num_folds -> number of partitions 
"""
def make_partitions(num_folds, data):
    data_size = len(data)
    fold_size = data_size // num_folds
    partitions = []
    for idx in range(num_folds-1):
        partitions.append(data[idx*fold_size:(idx+1)*fold_size])
    partitions.append(data[idx*fold_size:])
    return partitions

In [32]:
"""
    Returns the clique of the user as np.array

    user -> userID
    utility_matrix -> {user:{item:rating}}
    clique_size -> size of the clique of the user
"""
def compute_clique_without_implicit(user, utility_matrix, clique_size):

    unique_users = list(set(utility_matrix.keys()))
    user_dict = utility_matrix[user]
    partitions = make_partitions(CORES,unique_users)    
        

    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                user_dict = user_dict, 
                                                user_ids = partitions[idx], 
                                                utility_matrix = utility_matrix, 
                                                is_explicit = True)
        threads.append(thread)

    
    [thread.start() for thread in threads]    
    print('======= all threads started =======')

    results = [thread.join() for thread in threads]
    print('======= all threads joined =======')
    
    similarities = []
    for elem in results:
        for similarity in elem:
            similarities.append(similarity)
    
    similarities = np.array(similarities)
    clique = similarities[np.argsort(similarities[:,1])[::-1]][1:clique_size+1]

    return clique

In [33]:
"""
    weighted sum of scores of v1 and v2    

    v1 -> list [[ID, score]]
    v2 -> list [[ID, score]] 

    a, b weights
"""
def sum_vectors(v1, v2, a, b):
    ret = []

    for idx in range(len(v1)):
        if(v1[idx][0]!=v2[idx][0]):
            print('The two vectors has to have the same order of users!\nIf you see this message than you should change the return of ComputeCorrelationCoefficentThread.join() into a dictionary so that I can access the implicit and explicit scores for the same user with a small cost')
            raise Exception('The two vectors must have the same order of users!')
        
        summed_sim = v1[idx][1]*a + v2[idx][1]*b
        ret.append([v1[idx][0], summed_sim])
        
    print(ret)
    return ret

In [34]:
def compute_clique_with_implicit(user, esplicit_utility_matrix, implicit_utility_matrix, clique_size, a, b):

    unique_users = list(set(esplicit_utility_matrix.keys()))
    user_dict = esplicit_utility_matrix[user]
    partitions = make_partitions(CORES,unique_users)    
        
    print('======= start creating esplicit threads =======')

    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                user_dict = user_dict, 
                                                user_ids = partitions[idx], 
                                                utility_matrix = esplicit_utility_matrix,
                                                is_explicit = True)
        threads.append(thread)
    
    [thread.start() for thread in threads]    
    print('======= all esplicit threads started =======')

    results = [thread.join() for thread in threads]
    print('======= all esplicit threads joined =======')
    
    esplicit_similarities = []
    for elem in results:
        for similarity in elem:
            esplicit_similarities.append(similarity)
    
    print('======= start creating implicit threads =======')

    user_dict = implicit_utility_matrix[user]
    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx+CORES)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                user_dict = user_dict, 
                                                user_ids = partitions[idx], 
                                                utility_matrix = implicit_utility_matrix,
                                                is_explicit = False)
        threads.append(thread)
    
    [thread.start() for thread in threads]    
    print('======= all implicit threads started =======')

    results = [thread.join() for thread in threads]
    print('======= all implicit threads joined =======')
    
    implicit_similarities = []
    for elem in results:
        for similarity in elem:
            implicit_similarities.append(similarity)

    similarities = sum_vectors(esplicit_similarities, implicit_similarities, a, b)

    similarities = np.array(similarities)
    clique = similarities[np.argsort(similarities[:,1])[::-1]][1:clique_size+1]

    return clique




In [35]:
"""
    returns the predicted rating given a user and an item

    user -> userID
    item -> itemID
    clique -> list of users that are similar to userID
"""

def predict(user, item, clique, utility_matrix):
    numerator = 0
    denominator = 0
    for elem in clique:
        neighbor = elem[0]
        similarity = elem[1]

        neigh_dict = utility_matrix[neighbor]
        rating = neigh_dict.get(item)
        
        if  rating == None:
            continue

        numerator += rating*similarity
        denominator += similarity
    
    if denominator == 0:
        print('denominator is 0, no user in the clique rated this item')
        return -1
    
    return numerator/denominator
        

In [36]:
user = list(test_dict.keys())[0]    
print('user:{}'.format(user))

#clique = compute_clique_without_implicit(user, train_dict, 100) without implicit

clique = compute_clique_with_implicit(user, train_dict_explicit, train_dict_implicit, 100, 1, 1) #TODO define a and b in a proper manner
 

340050.0, 0.0], [1372820.0, 0.0], [1241750.0, 0.0], [2388640.0, 0.0], [1274530.0, 0.0], [1438370.0, 0.0], [815780.0, 0.0], [2093730.0, 0.0], [586410.0, 0.0], [357040.0, 0.0], [1045170.0, 0.0], [2093750.0, 0.0], [2421430.0, 0.0], [2781880.0, 0.0], [1700540.0, 0.0], [1798850.0, 0.0], [1012420.0, 0.0], [586440.0, 0.0], [750280.0, 0.0], [160460.0, 0.0], [357070.0, 0.0], [2650830.0, 0.0], [717520.0, 0.0], [2192080.0, 0.0], [2192090.0, 0.0], [2257630.0, 0.0], [979680.0, 0.0], [684780.0, 0.0], [2224880.0, 0.0], [2781940.0, 0.0], [1209080.0, 0.0], [1536760.0, 0.0], [2585340.0, 0.0], [2323200.0, 0.0], [2487040.0, 0.0], [1602310.0, 0.0], [979720.0, 0.0], [1864460.0, 0.0], [2061070.0, 0.0], [848660.0, 0.0], [389910.0, 0.0], [553750.0, 0.0], [2224920.0, 0.0], [2093850.0, 0.0], [1012510.0, 0.0], [1897250.0, 0.0], [2061090.0, 0.0], [324400.0, 0.0], [652080.0, 0.0], [1635120.0, 0.0], [881460.0, 0.0], [2028340.0, 0.0], [1733430.0, 0.0], [357180.0, 0.0], [2159420.0, 0.0], [389950.0, 0.0], [2192190.0, 0

In [38]:
test_items = list(test_dict[user].keys())

#predictions = np.array([ [item, predict(user, item, clique, train_dict), test_dict[user][item]] for item in test_items])
predictions = np.array([ [item, predict(user, item, clique, train_dict_explicit), test_dict[user][item]] for item in test_items])
print(predictions)

[[3.12252617e+09 8.19595672e+00 8.00000000e+00]
 [3.12261594e+09 7.98326267e+00 8.00000000e+00]
 [4.46677450e+09 9.30793171e+00 1.00000000e+01]
 [4.51166892e+09 6.16150924e+00 3.00000000e+00]]
